In [1]:
import json
import pandas as pd
import requests

In [7]:
df = pd.read_csv('resale-flat-prices-based-on-approval-date-1990-2012.csv')

In [8]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0
...,...,...,...,...,...,...,...,...,...,...
656846,2012-02,YISHUN,5 ROOM,212,YISHUN ST 21,10 TO 12,121.0,Improved,1985,476888.0
656847,2012-02,YISHUN,5 ROOM,758,YISHUN ST 72,01 TO 03,122.0,Improved,1986,490000.0
656848,2012-02,YISHUN,5 ROOM,873,YISHUN ST 81,01 TO 03,122.0,Improved,1988,488000.0
656849,2012-02,YISHUN,EXECUTIVE,664,YISHUN AVE 4,07 TO 09,181.0,Apartment,1992,705000.0


In [9]:
df_address = df['block'] + " " + df['street_name']

df_unique_address = df_address.drop_duplicates()
df_unique_address.reset_index(drop=True, inplace=True)
len(df_unique_address)

8416

In [10]:
df_unique_address

0         309 ANG MO KIO AVE 1
1         216 ANG MO KIO AVE 1
2         211 ANG MO KIO AVE 3
3         202 ANG MO KIO AVE 3
4         235 ANG MO KIO AVE 3
                 ...          
8411        79A TOA PAYOH CTRL
8412      25 NEW UPP CHANGI RD
8413    677A JURONG WEST ST 64
8414           633A PUNGGOL DR
8415           636A PUNGGOL DR
Length: 8416, dtype: object

In [19]:
# This cell ran for >30 min to retrieve 8k+ records

latitude = []
longitude = []
postal_code = []
full_address = []
missing = []

for row in range(len(df_unique_address)):
    query_address = df_unique_address[row]
    query_string = 'https://developers.onemap.sg/commonapi/search?searchVal=' + str(query_address) + '&returnGeom=Y&getAddrDetails=Y'
    resp = requests.get(query_string)

    # Convert JSON into Python Object
    data_geo_location = json.loads(resp.content)
    if data_geo_location['found'] != 0:
        latitude.append(data_geo_location['results'][0]['LATITUDE'])
        longitude.append(data_geo_location['results'][0]['LONGITUDE'])
        postal_code.append(data_geo_location['results'][0]['POSTAL'])
        full_address.append(data_geo_location['results'][0]['ADDRESS'])
    else:
        missing.append(query_address)

In [23]:
missing

['309 ANG MO KIO AVE 1',
 '247 ANG MO KIO AVE 3',
 '252 ANG MO KIO AVE 4',
 '455 ANG MO KIO AVE 10',
 '4 HILLVIEW AVE',
 '6 HILLVIEW AVE',
 '2 HILLVIEW AVE',
 '33 TAMAN HO SWEE',
 '114 BT MERAH VIEW',
 '18 KG BAHRU HILL',
 '29 HAVELOCK RD',
 '111 BT MERAH VIEW',
 '22 OUTRAM PK',
 '12 SHORT ST',
 '13 SHORT ST',
 '314 CLEMENTI AVE 4',
 '313 CLEMENTI AVE 4',
 '401 CLEMENTI AVE 1',
 '1 JLN PASAR BARU',
 '59 SIMS DR',
 '3 TEBAN GDNS RD',
 '5 TEBAN GDNS RD',
 '10 TEBAN GDNS RD',
 '8 YUNG KUANG RD',
 '10 YUNG KUANG RD',
 '145 HU CHING RD',
 '167 BOON LAY DR',
 '168 BOON LAY DR',
 '169 BOON LAY DR',
 '170 BOON LAY DR',
 '172 BOON LAY DR',
 '220 BOON LAY AVE',
 "2 ST. GEORGE'S RD",
 '20 UPP BOON KENG RD',
 "20 ST. GEORGE'S RD",
 "23 ST. GEORGE'S RD",
 '53 TANGLIN HALT RD',
 '54 TANGLIN HALT RD',
 '96 MARGARET DR',
 '24 DOVER CRES',
 '173 STIRLING RD',
 '172 STIRLING RD',
 '52 TANGLIN HALT RD',
 "74 C'WEALTH DR",
 "77 C'WEALTH DR",
 "79 C'WEALTH DR",
 "78 C'WEALTH DR",
 '5 SELETAR WEST FARMWAY 6

In [24]:
print(len(latitude), len(longitude), len(postal_code), len(full_address))

8282 8282 8282 8282


In [25]:
df_results = pd.DataFrame(list(zip(latitude, longitude, postal_code, full_address)), columns =['Latitude', 'Longitude', 'Postal Code', 'Full Address'])

In [26]:
df_results

,Latitude,Longitude,Postal Code,Full Address
0,1.366196788,103.841505,560216,216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPOR...
1,1.3691969659999998,103.8416666,560211,211 ANG MO KIO AVENUE 3 HDB-ANG MO KIO SINGAPO...
2,1.36844644,103.8445163,560202,202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAP...
3,1.366823609,103.8364912,560235,235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...
4,1.3683460680000001,103.83719599999999,560232,232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW S...
...,...,...,...,...
8277,1.334531295,103.8475657,311079,79A TOA PAYOH CENTRAL CENTRAL HORIZON SINGAPOR...
8278,1.323250133,103.9338356,462025,25 NEW UPPER CHANGI ROAD SINGAPORE 462025
8279,1.345630476,103.705038,641677,677A JURONG WEST STREET 64 EDELWEISS@JURONG SI...
8280,1.401369675,103.91319159999999,821633,633A PUNGGOL DRIVE EDGEDALE GREEN SINGAPORE 82...


In [28]:
df_results.to_csv('address_details.csv', index=False)